In [7]:
# Goal: turn the NIO.owl into the .xml format required as the input of the ConceptMapper

## There are two types of input formats -- the <AbbreviatedIRI> tag and <IRI> tag
# # input format -- first type:
#     <AnnotationAssertion>
#         <AnnotationProperty abbreviatedIRI="metadata:prefixIRI"/>
#         <AbbreviatedIRI>obo:NBO_0000304</AbbreviatedIRI>
#         <Literal datatypeIRI="http://www.w3.org/2001/XMLSchema#string">NBO:0000304</Literal>
#     </AnnotationAssertion>
#     <AnnotationAssertion>
#         <AnnotationProperty abbreviatedIRI="rdfs:label"/>
#         <AbbreviatedIRI>obo:NBO_0000304</AbbreviatedIRI>
#         <Literal datatypeIRI="http://www.w3.org/2001/XMLSchema#string">memory loss behavior</Literal>
#     </AnnotationAssertion>
#     <AnnotationAssertion>
#         <AnnotationProperty abbreviatedIRI="obo:synonym"/>
#         <AbbreviatedIRI>obo:NBO_0000304</AbbreviatedIRI>
#         <Literal datatypeIRI="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">forgetting</Literal>
#     </AnnotationAssertion>
# ps. The "obo:synonym" can also be "oboInOwl:hasExactSynonym", "oboInOwl:hasNarrowSynonym", "NDDUO:Synonym", 
# "oboInOwl:hasBroadSynonym", "oboInOwl:hasRelatedSynonym", 

# # expected format -- This format is not right. The canonical has to be in the variant.
# <?xml version="1.0" ?>
# <synonym>
#     <token id = "ncbi xxxxx", canonical="colon, nos">
#          <variant base="large intestine"/>
#          <variant base="large bowel"/>
#     </token>
# </synonym>

# # expected format:
# <?xml version="1.0" ?>
# <synonym>
#     <token id = "ncbi xxxxx", canonical="colon, nos">
#          <variant base="colon, nos"/>
#          <variant base="large intestine"/>
#          <variant base="large bowel"/>
#     </token>
# </synonym>


In [1]:
nio_path = "/Users/yidesdo21/Projects/inputs/ontologies/NIO1.1.owl"

In [2]:
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
from xml.dom import minidom

In [3]:
from nltk.stem import WordNetLemmatizer

In [11]:
# import nltk
# nltk.download('wordnet')

In [4]:
with open(nio_path) as f:
    contents = f.read()
#     print(contents)

In [5]:
soup = BeautifulSoup(contents, 'html.parser')

In [14]:
# soup.find_all('annotationassertion')

### sanity check for the owl file

In [15]:
# ## check for the structure of the owl
# # all tags
# set([tag.name for tag in soup.find_all()])

In [16]:
# # check the distinct <AnnotationProperty abbreviatedIRI= ...> lines 
# # check the combinations of tags and attributes, such as:
# # <annotationassertion>
# #  <annotationproperty abbreviatediri="rdfs:label"></annotationproperty>  <---- the "rdfs:label" attribute can also be "obo:synonym"
# #  <iri>...</iri>    <---- the <iri> tag can also be <abbreviatediri>
# #  <literal datatypeiri="http://www.w3.org/2001/XMLSchema#string">1.1.1</literal>
# # </annotationassertion>,

# property_attris = set()
# attribute_assert = defaultdict(list)
# attributes = ['metadata:prefixIRI', 'rdfs:label', "obo:synonym", 
#               "oboInOwl:hasExactSynonym", "oboInOwl:hasNarrowSynonym", 
#               "NDDUO:Synonym", "oboInOwl:hasBroadSynonym", "oboInOwl:hasRelatedSynonym"]
# tags = ["annotationproperty", "abbreviatediri", "iri", "literal"]


# for anno_assert in soup.find_all('annotationassertion'):  
#     property_tag = anno_assert.annotationproperty   
#     property_attri = property_tag["abbreviatediri"]    # str
# #     iri_url = anno_assert.iri 

# #     # check the distinct property attributes, overall 79
# #     property_attris.add(property_attri)
    
#     if property_attri in attributes:
#         attribute_assert[property_attri].append(anno_assert)
# #         print(anno_assert)
# #         break
# #         print("--------")
    
# #     # only look at the <IRI> structure, excluding the <AbbreviatedIRI>
# #     if iri_url is not None:
# #         if property_attri in ["metadata:prefixIRI", "NDDUO:Synonym", "rdfs:label"]:   # have checked for the "NDDUO:Synonym", this is the only attribute that has the synonyms
# # #         if property_attri == "metadata:prefixIRI":   # check if the guess of using <IRI> as the token id is true for those who are not from ADO. the answer is no 
# #             url = iri_url.contents[0]
# #             split_url = url.split("/")

In [17]:
# cnt = 0
# test_dict = defaultdict(list)
# abbre_iri_dict = defaultdict(list)
# abbre_cnt, iri_cnt = 0,0
# other, ado = 0,0
# test_id = 0
# synonoyms = ["obo:synonym", "oboInOwl:hasExactSynonym", "oboInOwl:hasNarrowSynonym", 
#               "NDDUO:Synonym", "oboInOwl:hasBroadSynonym", "oboInOwl:hasRelatedSynonym"]

# for syn in synonoyms:
# #     print(syn)
#     if syn == "oboInOwl:hasBroadSynonym":
#         continue
    
#     for anno_assert in attribute_assert.get(syn):
# #         print(anno_assert)
#     #     print(anno_assert.contents[3].name)
#     #     print(anno_assert.contents)
#         cnt += 1
#         abbre_or_iri = anno_assert.contents[3] # use .content to access tags (children) inside the tag
#     #     print(abbre_or_iri)   
#     #     print(abbre_or_iri.name)
#         if abbre_or_iri.name == "abbreviatediri":
#             abbre_cnt += 1
#             split_text = abbre_or_iri.get_text().split("/")
            
#             if len(split_text) > 1:
#                 print(split_text)
#                 if split_text[-1].split("#")[0] == "OntoDT" or split_text[-2] == "OntoDT":
#                     print(anno_assert)
#                     print("------")
                
#         elif abbre_or_iri.name == "iri":
#             iri_cnt += 1
#     #         print(abbre_or_iri)
#             split_text = abbre_or_iri.get_text().split("/")
#     #         if split_text[-2] == "ppmi":
#     #             test_id += 1
#             if len(split_text) > 1:
# #                 print(split_text)
#                 if split_text[-1].split("#")[0] == "OntoDT" or split_text[-2] == "OntoDT":
#         #         if len(split_text) > 5 and split_text[4] == "http:":
#         #                 test_id += 1
#         #                 print(split_text)
#                     print(anno_assert)
#                     print("------")
#     # #             print("true")
#     #         print(split_text)
#     #         print("------")
# #             if len(split_text) <= 1:
# #                 print(anno_assert)
# #                 print("-------")
# #                 other += 1
#     #         else:
#     #             ado += 1

#     #         print(anno_assert.contents[-2].get_text())
#     #         print("-------")

#     #     abbre_or_iri = anno_assert.contents[3].get_text().split(":")[-1].split("_")
#     #     test_dict[abbre_or_iri[0]].append(abbre_or_iri[1])
#     #     print(anno_assert.contents[-2].get_text())   # the content of the <literal> tag, 
# #         print("--------------------❤️")

# #     print(other)
#     # print(cnt)

In [18]:
## combinations of tags and attributes
# 1. attribute-"rdfs:label" + tags-<iri>, <abbreviatediri>, 3922 cases
# 1.1 "rdfs:label" + <iri>: <iri> has url, need to be processed for token id, literal has canonical, 625 cases
# 1.1.1 different ways to process token id in the <iri> tag
# OntoDT, 32 cases
#  <annotationassertion>
#     <annotationproperty abbreviatediri="rdfs:label"></annotationproperty>
#     <iri>http://ontodm.com/OntoDT#OntoDT_inst_507131</iri>
#     <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">OntoDT: field identifier</literal>
#  </annotationassertion>,

# 1.1.2 obo-fma, 6
# <iri>http://purl.obolibrary.org/obo/fma#attaches_to</iri>
# attaches_to    <--- this is the content in the literal tag

# 1.1.3 NPT, 3
# <iri>http://purl.obolibrary.org/obo/NPT.owl#test_images</iri>
# test images

# 1.1.4 OBO-behavior, 21
# <iri>http://purl.obolibrary.org/obo/behavior.owl/NBO_0002379</iri>
# nursing behavior

# 1.1.5 ADO, 410
# <iri>http://scai.fraunhofer.de/aa</iri>
# A2M

# 1.1.6 ADO-NINCDS, 1
# <iri>http://scai.fraunhofer.de/AlzheimerOntology#NINCDS/ADRDA_criteria</iri>
# NINCDS/ADRDA criteria

# 1.1.7 BFO-SNAP/SPAN, 9
# <iri>http://www.ifomis.org/bfo/1.1/snap#DependentContinuant</iri>
# dependent_continuant

# 1.1.8 OntoDT-DMtypes, 8, unlabeled data, dump these
# <iri>http://www.ontodm.com/OntoDT/DMtypes/DMType_00a74a71_fcf5_45a0_853b_eeec6b0ee020</iri>
# OntoDT: unlabeled data with missing values

# 1.1.9 NDDO, 2
# <iri>http://www.purl.org/NDDO/Montreal_Cognitive_Assessment__(MoCA)_design</iri>
# Montreal Cognitive Assessment (MoCA) design

# 1.1.10 NDDO-adni/ppmi, adni-22, ppmi-30
# <iri>http://www.purl.org/NDDO/adni/Boston_Naming_Test_design</iri>
# Boston Naming Test design

# 1.1.11 NDDO-http, 2
# <iri>http://www.purl.org/NDDO/http://purl.obolibrary.org/obo/OBI_0000658</iri>
# OBI: data representational model
# <annotationassertion>
#   <annotationproperty abbreviatediri="rdfs:label"></annotationproperty>
#   <iri>http://www.purl.org/NDDO/http://ontodm.com/OntoDT#OntoDT_184436</iri>
#   <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral" xml:lang="en">OntoDT: identifier</literal>
# </annotationassertion>
# ------
# <annotationassertion>
#   <annotationproperty abbreviatediri="rdfs:label"></annotationproperty>
#   <iri>http://www.purl.org/NDDO/http://purl.obolibrary.org/obo/OBI_0000658</iri>
#   <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">OBI: data representational model</literal>
# </annotationassertion>


# 1.1.12 these canonicals don't point to any token ids, dump these, 79
# <iri>#Amnestic_Mild_Cognitive_Impairment</iri>
# Amnestic Mild Cognitive Impairment


# 1.2 "rdfs:label" + <abbreviatediri>: <abbreviatediri> has token id, <literal> has canonical , 3297 cases
#  <annotationassertion>
#     <annotationproperty abbreviatediri="rdfs:label"></annotationproperty>
#     <abbreviatediri>obo:BFO_0000001</abbreviatediri>
#     <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral" xml:lang="en">entity</literal>
#  </annotationassertion>,



In [19]:
# 2. attribute-"metadata:prefixIRI" + tag-<iri>, <abbreviatediri>
# overall 2025 cases
# 2.1 abbreviatediri, 1552 cases
#  <annotationassertion>
#     <annotationproperty abbreviatediri="metadata:prefixIRI"></annotationproperty>
#     <abbreviatediri>obo:BFO_0000001</abbreviatediri>
#     <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">BFO:0000001</literal>
#  </annotationassertion>,

# 2.2 iri, 473 cases
# 2.2.1 the <iri> structure for AlzheimerOntology, 408 cases, 2 duplicates
# <iri>http://scai.fraunhofer.de/AlzheimerOntology#A2M</iri>
# AlzheimerOntology:A2M

# 2.2.2 for the SNAP/SPAN ontology in BFO, 2 for snap, 7 for span
# <iri>http://www.ifomis.org/bfo/1.1/snap#DependentContinuant</iri>
# snap:DependentContinuant

# 2.2.3 for NDDO, 2 cases
# <iri>http://www.purl.org/NDDO/Montreal_Cognitive_Assessment__(MoCA)_design</iri>
# NDDO:_design    <---- this is not right even in the owl file, should be NDDO:Montreal_Cognitive_Assessment__(MoCA)_design
    
# 2.2.4 for the Alzheimer's Disease Neuroimaging (ADNI) or ppmi dataset, 22 for adni, 30 for ppmi
# <iri>http://www.purl.org/NDDO/adni/Clock_Drawing_Assessment_design</iri>
# Clock_Drawing_Assessment_design   <---- this is not right, should be NDDO:adni_Clock_Drawing_Assessment_design
#                                  better to use the <iri> tag as the token id rather than the <literal> tag's content


In [20]:
# 3. attribute-"obo:synonym"+tag<abbreviatediri>
# all for token id from the ontology -- obo:NBO_..., 203 cases
# <abbreviatediri>obo:NBO_0000003</abbreviatediri>
# affective behaviour

# 4. "oboInOwl:hasExactSynonym"+<abbreviatediri>
# overall 471 cases
# obo:GO_... -- 85 or obo:FMA_... -- 386   
# <abbreviatediri>obo:FMA_224850</abbreviatediri>
# Prefrontal association cortex

# 5. "oboInOwl:hasNarrowSynonym"+<abbreviatediri>   
# all for obo:GO_..., only 6 cases
# <abbreviatediri>obo:GO_0007631</abbreviatediri>
# drinking

# 6. the attribute as "oboInOwl:hasBroadSynonym" doesn't exist

# 7. "oboInOwl:hasRelatedSynonym"+<abbreviatediri> 
# all for obo:GO_..., only 4 cases
# <abbreviatediri>obo:GO_0043050</abbreviatediri>
# pumping behavior

# 8. "NDDUO:Synonym"+ <iri>or<abbreviatediri>
# overall 562 cases, 439 from <iri> tags, 123 from <abbreviatediri> tags
# 8.1 "NDDUO:Synonym"+ <iri>
# 8.1.1 for the AlzheimerOntology, 412 cases
# <annotationassertion>
#    <annotationproperty abbreviatediri="NDDUO:Synonym"></annotationproperty>
#    <iri>http://scai.fraunhofer.de/AlzheimerOntology#ADCS_ADL</iri>
#    <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">Alzheimer's Disease Cooperative Study - Activities of Daily Living
# </literal>
# </annotationassertion>

# 8.1.2 27 cases, unclear about the token id for the synonym, 
# the iri is also not linked to any token id, dump these cases
# <annotationassertion>
#    <annotationproperty abbreviatediri="NDDUO:Synonym"></annotationproperty>
#    <iri>#House_and_cube</iri>
#    <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral" xml:lang="en">Cube house</literal>
# </annotationassertion>


### extract 

In [21]:
# ## helper function
# def lower_variants(token):
#     """lowercase and ... the canonical and variants before adding them to the dictionary"""
#     pass

In [6]:
## deal with the <AbbreviatedIRI> tag input format: 
# <AnnotationAssertion>
#     <AnnotationProperty abbreviatedIRI="rdfs:label"/>
#     <AbbreviatedIRI>obo:OBI_0002193</AbbreviatedIRI>
#     <Literal xml:lang="en" datatypeIRI="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">Thermo Fisher Scientific</Literal>
# </AnnotationAssertion>

# wnl = WordNetLemmatizer()
token_ids = set()
id_canonicals = list()
id_variants = list()
syns = ["obo:synonym", "oboInOwl:hasExactSynonym", "oboInOwl:hasNarrowSynonym", 
        "NDDUO:Synonym", "oboInOwl:hasBroadSynonym", "oboInOwl:hasRelatedSynonym"]
token_ids_meta, token_ids_label = list(), list()

for anno_assert in soup.find_all('annotationassertion'):
    property_tag = anno_assert.annotationproperty
    
    # property_attri extracts "rdfs:label" in <AnnotationProperty abbreviatedIRI="rdfs:label"/>
    # it can also be <AnnotationProperty abbreviatedIRI="obo:synonym"/>, etc. 
    property_attri = property_tag["abbreviatediri"]    # str
    
    # iri_tag extracts "obo:OBI_0002193" in <AbbreviatedIRI>obo:OBI_0002193</AbbreviatedIRI>
    iri_tag = anno_assert.abbreviatediri   # the iri tag. type(iri_tag) == Tag
    
    # only look at the <AbbreviatedIRI> structure, exclusing the <IRI>
    if iri_tag is not None:
        token_id = iri_tag.contents[0]
        
        if property_attri == "metadata:prefixIRI":  # point to the token id ## the number is smaller than the token_ids in the labels
            token_ids_meta.append(token_id)

        # extracts the (token_id, canonical) pair from the following structure
        # <AnnotationAssertion>
        #     <AnnotationProperty abbreviatedIRI="rdfs:label"/>
        #     <AbbreviatedIRI>obo:OBI_0002193</AbbreviatedIRI>
        #     <Literal xml:lang="en" datatypeIRI="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">Thermo Fisher Scientific</Literal>
        # </AnnotationAssertion>    
        if property_attri == "rdfs:label":   # point to the canonical
            token_ids.add(token_id)   # # point to the token id
            canonical = anno_assert.literal.get_text()
#             canonical = anno_assert.literal.get_text().lower() 
            id_canonicals.append((token_id, canonical))
#             id_canonicals.append((token_id, wnl.lemmatize(canonical)))  # has duplications, why?
            token_ids_label.append(token_id)
            # using set(id_canonicals) can filter the duplications
#             print(anno_assert)
#             print("----------")
        
        # extracts the (token_id, variant) pair from the following structure
        # <AnnotationAssertion>
        #     <AnnotationProperty abbreviatedIRI="obo:synonym"/>
        #     <AbbreviatedIRI>obo:NBO_0000459</AbbreviatedIRI>
        #     <Literal datatypeIRI="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">vigilance</Literal>
        # </AnnotationAssertion>        
        if property_attri in syns:  # point to the variant base
            variant = anno_assert.literal.get_text()
#             variant = anno_assert.literal.get_text().lower() 
            id_variants.append((token_id, variant))
#             id_variants.append((token_id, wnl.lemmatize(variant)))
    
token_ids = sorted(list(token_ids))


In [7]:
len(token_ids)

3243

In [8]:
len(token_ids_meta)

1552

In [9]:
len(token_ids_label)

3297

In [10]:
# the token ids in the "metadata:prefixIRI" attribute but not in the "rdfs:label" attribute
set(token_ids_meta) - set(token_ids_label) 

{'owl:Thing'}

In [11]:
# the token ids in the "rdfs:label" attribute but not in the "metadata:prefixIRI" attribute
set(token_ids_label) - set(token_ids_meta)

{'obo:OBI_0000767',
 'obo:OBI_0002247',
 'obo:FMA_83902',
 'obo:FMA_70673',
 'obo:NPT_0011064',
 'obo:FMA_72780',
 'obo:FMA_72763',
 'obo:FMA_72740',
 'obo:FMA_70694',
 'obo:NPT_0010036',
 'obo:NPT_0020033',
 'obo:OBI_0002340',
 'obo:IAO_0000604',
 'obo:NPT_0020018',
 'obo:NPT_0020171',
 'obo:OBI_0002255',
 'obo:FMA_258685',
 'obo:BFO_0000082',
 'obo:OBI_0000768',
 'obo:NPT_0012072',
 'obo:FMA_62430',
 'obo:FMA_58817',
 'obo:NPT_0010017',
 'obo:BFO_0000066',
 'obo:OBI_0002207',
 'obo:FMA_71038',
 'obo:OBI_0002328',
 'obo:FMA_71037',
 'obo:OBI_0002263',
 'obo:FMA_71065',
 'obo:FMA_77700',
 'obo:OBI_0000769',
 'obo:NPT_0020091',
 'obo:FMA_70678',
 'obo:OBI_0002330',
 'obo:OBI_0001856',
 'obo:NPT_0010005',
 'obo:FMA_68628',
 'obo:RO_0000052',
 'obo:IAO_0000651',
 'obo:OBI_0002313',
 'obo:NPT_0020084',
 'obo:OBI_0001458',
 'obo:FMA_67812',
 'obo:FMA_72739',
 'obo:BFO_0000067',
 'obo:OBI_0000761',
 'obo:FMA_258696',
 'obo:FMA_72777',
 'obo:IAO_0000118',
 'obo:FMA_70692',
 'obo:IAO_0000234',

In [12]:
dup_pair = list()
for pair, cnt in Counter(id_canonicals).items():
    if cnt > 1:
        dup_pair.append((pair,cnt))

In [13]:
len(Counter(id_canonicals))

3263

In [14]:
dup_pair

[(('obo:BFO_0000015', 'process'), 2),
 (('obo:BFO_0000050', 'isPartOf'), 2),
 (('obo:BFO_0000051', 'hasPart'), 2),
 (('obo:BFO_0000055', 'realizes'), 2),
 (('obo:BFO_0000066', 'occurs in'), 2),
 (('obo:BFO_0000110', 'has continuant part at all times'), 2),
 (('obo:BFO_0000176', 'part of continuant at some time'), 2),
 (('obo:IAO_0000111', 'editor preferred label'), 2),
 (('obo:IAO_0000111', 'editor preferred term'), 2),
 (('obo:IAO_0000115', 'definition'), 3),
 (('obo:IAO_0000115', 'textual definition'), 2),
 (('obo:IAO_0000116', 'editor note'), 2),
 (('obo:IAO_0000119', 'definition source'), 2),
 (('obo:IAO_0000232', 'curator note'), 2),
 (('obo:NPT_0011015', 'assay design'), 2),
 (('obo:RO_0000056', 'participatesIn'), 2),
 (('obo:RO_0000057', 'hasParticipant'), 2),
 (('obo:RO_0000059', 'concretizes'), 2),
 (('dc:contributor', 'Contributor'), 2),
 (('dc:coverage', 'Coverage'), 2),
 (('dc:creator', 'Creator'), 2),
 (('dc:date', 'Date'), 2),
 (('dc:description', 'Description'), 2),
 (('

In [31]:
# duplications in token id and canonical 
## reason 1 -- same token id, same canonical, different datatypeiri
# <annotationassertion>
# <annotationproperty abbreviatediri="rdfs:label"></annotationproperty>
# <abbreviatediri>obo:BFO_0000015</abbreviatediri>
# <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral" xml:lang="en">process</literal>
# </annotationassertion>
# ----------
# <annotationassertion>
# <annotationproperty abbreviatediri="rdfs:label"></annotationproperty>
# <abbreviatediri>obo:BFO_0000015</abbreviatediri>
# <literal datatypeiri="http://www.w3.org/2001/XMLSchema#string">process</literal>
# </annotationassertion>

# how I solve this: filter the duplications, the token id -- caconical pair reduces from 3297 -> 3263

In [32]:
# duplications in token id
## same token id, different caconical, same datatypeiri
# e.g. 
# <annotationassertion>
# <annotationproperty abbreviatediri="rdfs:label"></annotationproperty>
# <abbreviatediri>obo:BFO_0000024</abbreviatediri>
# <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral" xml:lang="en">fiat object</literal>
# </annotationassertion>
# ----------
# <annotationassertion>
# <annotationproperty abbreviatediri="rdfs:label"></annotationproperty>
# <abbreviatediri>obo:BFO_0000024</abbreviatediri>
# <literal datatypeiri="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral" xml:lang="en">fiat object part</literal>
# </annotationassertion>
## why this is the case: The NIO has 3303 classes, i.e., class id, e.g. http://purl.obolibrary.org/obo/BFO_0000024
## for some class id, it has more than one "BFO OWL specification label", which is the same for the "label"
## e.g. for the class id http://purl.obolibrary.org/obo/BFO_0000024, it has two BFO OWL specification label: 
## 1) fiat object part, 2) fiat object. So it has two labels. When BFO_0000024 is written in the owl format, it 
## becomes the same token id, but with different caconicals. 
# All the token ids (class id) that have more than one caconicals:
# http://ontodm.com/OntoDT#OntoDT_0000023 -- OntoDT: string datatype|StringDatatype
# http://purl.obolibrary.org/obo/BFO_0000015 -- process|process
# http://purl.obolibrary.org/obo/OBI_0500000 -- study design|study specification
# http://www.purl.org/NDDO/NDDO_10002233 -- Boston Naming Test Scale|Boston Naming Test
# http://purl.obolibrary.org/obo/BFO_0000024 -- fiat object part|fiat object
# http://scai.fraunhofer.de/AlzheimerOntology#Late_Onset_Alzheimer_s_Disease -- Late Onset Alzheimer's Disease|late-onset AD
# http://scai.fraunhofer.de/AlzheimerOntology#CPS -- complex partial seizures|complex partial seizure
# http://purl.obolibrary.org/obo/IAO_0000033 -- directive information entity|directive information content entity
# http://www.purl.org/NDDO/NDDO_10003203 -- NPI Hallucinations Datatype|NPI Euphoria Datatype

## How to deal with this: the key is about how the .xml format will affect the ConceptWrapper
## will the same token ids with different caconicals be an issue? How about add one caconical as the variant?
## But using which one? It doens't have a preferred label mark in the owl file. 
## Antonio suggests to turn one canonical as the variant because if the token_id is not unique,
## "ConceptMapper will consider one of the two".

# how I solve this: turn the different canonicals for the same token id into a list
# we have a dictionary for token id -- canonicals: {"the same token id": ["canonical1", "canonical2"]}
# the token id -- canonical pair reduces from 3263 -> 3243
# turn one canonical as the variant


In [33]:
# using "metadata:prefixIRI" as a pointer to token_id is not sufficient
# if using "metadata:prefixIRI", there are 1552 unique token_ids, 
# but if using "rdfs:label", the number becomes 3243, this will make the number of elements in the .xml file 
# smaller than it is actually be.
## solution: use "rdfs:label" to identify both token_id and canonical

In [15]:
len(id_variants)

807

In [16]:
id_canonicals = list(set(id_canonicals))

In [17]:
len(id_canonicals)

3263

In [18]:
dict_id_canonicals = defaultdict(list)
for token_id, cano in id_canonicals:
    dict_id_canonicals[token_id].append(cano)

In [19]:
len(dict_id_canonicals)

3243

In [20]:
sorted(dict_id_canonicals.items())

[('NDDO:Geriatric_Depression_Scale_Assessment_design',
  ['Geriatric Depression Scale Assessment design']),
 ('NDDO:NDDO_00000021', ['visit']),
 ('NDDO:NDDO_00000022', ['diagnostic data']),
 ('NDDO:NDDO_00000023', ['MRI image']),
 ('NDDO:NDDO_00000024', ['clinical data']),
 ('NDDO:NDDO_00000025', ['biomarker data']),
 ('NDDO:NDDO_00000027', ['PET image']),
 ('NDDO:NDDO_00000030', ['image data score']),
 ('NDDO:NDDO_00000036', ['Telephone visit']),
 ('NDDO:NDDO_00000037', ['Physical visit']),
 ('NDDO:NDDO_00000040', ['hasAgent']),
 ('NDDO:NDDO_00000041', ['expressedIn']),
 ('NDDO:NDDO_00000042', ['extractedFrom']),
 ('NDDO:NDDO_00000043', ['hasDatatype']),
 ('NDDO:NDDO_00000044', ['hasID']),
 ('NDDO:NDDO_00000045', ['hasInformation']),
 ('NDDO:NDDO_00000046', ['hasInput']),
 ('NDDO:NDDO_00000047', ['hasOutput']),
 ('NDDO:NDDO_00000048', ['hasDate']),
 ('NDDO:NDDO_00000049', ['hasValue']),
 ('NDDO:NDDO_00000050', ['isOfGender']),
 ('NDDO:NDDO_00000051', ['isOutputOf']),
 ('NDDO:NDDO_0000

In [21]:
dict_id_variants = defaultdict(list)
for token_id, variant in id_variants:
    dict_id_variants[token_id].append(variant)

In [22]:
len(dict_id_variants)

469

In [23]:
dict_id_canonicals.get('owl:Thing', None)

In [43]:
# why the numbers of id_canonicals and id_variants are different?
# one token id can have multiple variants, the number of id_canonicals should be smaller than the id_variants
# what if one token id has multiple canonicals? 
## the above is not right, the the number of id_canonicals might be larger than the id_variants
## because not every caconical has variants



# you can have the token_id, but the token_id doesn't have the corresponded canonical
# e.g. for the token_id -- 'owl:Thing', we don't have a related canonical in the dict_id_canonicals['owl:Thing']
# The following the only place where 'owl:Thing' appears in the AnnotationAssertion
# It is more like the rubbish data, we don't want it. I dump the whole element (<token id =...> ... <variant ...>) 
# when it can't find a canonical with the token_id.
#     <AnnotationAssertion>
#         <AnnotationProperty abbreviatedIRI="metadata:prefixIRI"/>
#         <AbbreviatedIRI>owl:Thing</AbbreviatedIRI>
#         <Literal datatypeIRI="http://www.w3.org/2001/XMLSchema#string">owl:Thing</Literal>
#     </AnnotationAssertion>


# to be dealt with: duplicated token id and canonical -- dealt
# 	<token id="obo:BFO_0000031" canonical="generically dependent continuant"/>
#	<token id="obo:BFO_0000031" canonical="generically dependent continuant"/>

## The IRI tag input format

In [44]:
## deal with the <IRI> tag input format: 
## <IRI> circumstance 1: from the Alzheimer Ontology, the token id has two formats  --- this is outdated, see the sanity check part
##   1.1 AlzheimerOntology#brainstem   1.2 AlzheimerOntology#NINCDS/ADRDA_criteria
# <AnnotationAssertion>
#         <AnnotationProperty abbreviatedIRI="metadata:prefixIRI"/>   
#         <IRI>http://scai.fraunhofer.de/AlzheimerOntology#brainstem</IRI>   <----- for token id 
#      or <IRI>http://scai.fraunhofer.de/AlzheimerOntology#NINCDS/ADRDA_criteria</iri>   <---- splitting by "/" is not enough
#         <Literal datatypeIRI="http://www.w3.org/2001/XMLSchema#string">AlzheimerOntology:brainstem</Literal>
#     </AnnotationAssertion>
#     <AnnotationAssertion>
#         <AnnotationProperty abbreviatedIRI="NDDUO:Synonym"/>       <----- for synonym
#         <IRI>http://scai.fraunhofer.de/AlzheimerOntology#brainstem</IRI>  <----- for token id 
#         <Literal datatypeIRI="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">rostral brainstem</Literal>
#     </AnnotationAssertion>
#     <AnnotationAssertion>
#         <AnnotationProperty abbreviatedIRI="rdfs:comment"/>
#         <IRI>http://scai.fraunhofer.de/AlzheimerOntology#brainstem</IRI>
#         <Literal datatypeIRI="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">from manual annotation of text corpus</Literal>
#     </AnnotationAssertion>
#     <AnnotationAssertion>
#         <AnnotationProperty abbreviatedIRI="rdfs:label"/>    <----- for canonical
#         <IRI>http://scai.fraunhofer.de/AlzheimerOntology#brainstem</IRI>  <----- for token id 
#         <Literal datatypeIRI="http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral">brainstem</Literal>
# </AnnotationAssertion>

In [45]:
## <IRI> circumstance 2: from the BFO, the extraction is same to 1.1 
# <annotationassertion>
#    <annotationproperty abbreviatediri="metadata:prefixIRI"></annotationproperty>
#    <iri>http://www.ifomis.org/bfo/1.1/snap#DependentContinuant</iri>
#    <literal datatypeiri="http://www.w3.org/2001/XMLSchema#string">snap:DependentContinuant</literal>
# </annotationassertion>

In [46]:
## <IRI> circumstance 3: from the NDDO, totally different from ADO and BFO
# <annotationassertion>
#    <annotationproperty abbreviatediri="metadata:prefixIRI"></annotationproperty>
#    <iri>http://www.purl.org/NDDO/ppmi/Unified_Parkinsons_Disease_Rating_Scale_(UPRDS)_Assessment_design</iri>
#    <literal datatypeiri="http://www.w3.org/2001/XMLSchema#string">_Assessment_design</literal>
# </annotationassertion>

In [47]:
## a quick scan finds out that for the property attributes "metadata:prefixIRI",
#  the token ids come from Alzheimer Ontology, BFO, and NDDO. The latter two are very few compared to ADO,
#  but NDDO is structured very differently from ADO and BFO.
## for the property attributes "rdfs:label", it's a lot more than ADO. 
## what I am going to do: I will focus on the ADO only for now. 

In [24]:
## only for <IRI> 
ids_iri = list()   # this list shouldn't be used in later processing because it is incomplete, use id_canonical instead
id_cano_iri = list()
id_var_iri = list()
onto_matches = ["AlzheimerOntology", "OntoDT", "obo", "bfo", "NDDO"]

for anno_assert in soup.find_all('annotationassertion'):
    property_tag = anno_assert.annotationproperty
    
    # property_attri extracts "rdfs:label" in <AnnotationProperty abbreviatedIRI="rdfs:label"/>
    # it can also be <AnnotationProperty abbreviatedIRI="NDDUO:Synonym"/>, etc. 
    property_attri = property_tag["abbreviatediri"]    # str
    
    # tag_iri has a url, the last part is the token id (need preprocessing)
    # e.g. <IRI>http://scai.fraunhofer.de/AlzheimerOntology#brainstem</IRI>
    # replace the "#" sign as ":" in <AlzheimerOntology#brainstem>, then it becomes the token id
    iri_url = anno_assert.iri 
    
    # only look at the <IRI> structure, excluding the <AbbreviatedIRI>
    if iri_url is not None:
        if property_attri in ["metadata:prefixIRI", "NDDUO:Synonym", "rdfs:label"]:   # have checked for the "NDDUO:Synonym", this is the only attribute that has the synonyms
#         if property_attri == "metadata:prefixIRI":   # check if the guess of using <IRI> as the token id is true for those who are not from ADO. the answer is no 
            url = iri_url.contents[0]
            split_url = url.split("/")
        
            if len(split_url) > 3 and any(x in split_url[3] for x in onto_matches):   # dumped the unclear tokens
                rev_fir, rev_sec, third = split_url[-1], split_url[-2], split_url[3]
                len_split = len(split_url)
                if len_split == 4:  
                    token_id = rev_fir.replace("#", ":")  # For ADO and OntoDT
                    
                elif len_split == 5:   # only one AlzheimerOntology#NINCDS/ADRDA_criteria, three duplications  
                    if rev_sec == "AlzheimerOntology#NINCDS": # for ADO
                        token_id = (rev_sec+"_"+rev_fir).replace("#",":")   
                    elif rev_sec == "OntoDT" or rev_sec == "NDDO":  # for OntoDT and NDDO
                        token_id = rev_sec+":"+rev_fir
                    elif rev_sec == "obo":  # for obo-fma, obo-npt
                        token_id = rev_sec+":"+rev_fir.replace("#", "_").replace(".owl", "")

                elif len_split == 6: # for obo-bahavior, bfo, NDDO
                    if rev_sec == "DMtypes":   # exclude DMtypes, not in the OntoDT.owl
                        continue    
                    if third == "ofo":
                        token_id = third+":"+rev_fir
                    elif third == "bfo":
                        token_id = third+":"+rev_fir.replace("#", "_")
                    elif third == "NDDO":
                        token_id = third+":"+rev_sec+"_"+rev_fir
                        
                elif len_split == 8:  # for OntoDT#OntoDT_184436
                    token_id = rev_fir.replace("#", ":")
            
                    
                if property_attri == "metadata:prefixIRI":    # using this attribute to extract token ids is incomplete
                    ids_iri.append(token_id)
 
                if property_attri == "rdfs:label":   # point to the canonical
                    canonical = anno_assert.literal.get_text()
#                     canonical = anno_assert.literal.get_text().lower()
                    if token_id.split(":")[0] == "OntoDT":
                        cano_onto = canonical.split(":")
                        if len(cano_onto) > 1:
                            canonical = cano_onto[1].lstrip()
                    id_cano_iri.append((token_id, canonical))
#                     id_cano_iri.append((token_id, wnl.lemmatize(canonical)))  
           
                if property_attri == "NDDUO:Synonym":  # point to the variant base
                    variant = anno_assert.literal.get_text()
#                     variant = anno_assert.literal.get_text().lower()
                    if variant == "http://www.ebi.ac.uk/efo/efo_0000493":
                        continue
                    id_var_iri.append((token_id, variant))
#                     id_var_iri.append((token_id, wnl.lemmatize(variant)))
    


In [25]:
len(ids_iri)

473

In [26]:
len(set(ids_iri))

471

In [27]:
len(id_cano_iri)

538

In [28]:
len(id_var_iri)

412

In [29]:
len(dict_id_canonicals)

3243

In [30]:
## turn the ontologies with <IRI> tag into id_canonical and id_variant dictionaries
## add the ontologies to the existed dict_id_canonicals
token_id_iri = list()

for token_id, cano in id_cano_iri:
    token_id_iri.append(token_id)
    dict_id_canonicals[token_id].append(cano)
    
for token_id, variant in id_var_iri:
    dict_id_variants[token_id].append(variant)

In [31]:
len(dict_id_canonicals)

3755

In [32]:
len(dict_id_variants)

648

In [33]:
dict_id_canonicals.get("obo:NBO_0000114")

['aggressive behavior towards children']

In [34]:
dict_id_variants.get("obo:NBO_0000114")

['aggressive behavior towards children']

In [59]:
# step 2: link the token_ids, canonical, and variant base together



In [60]:
# from xml.dom import minidom
# import os

# users_list = ["GeeksForGeeks", "Arka", "Computer Science", "Engineering", "Portal"]

# # create file
# xmldoc = minidom.Document()

# # creat root element
# root_element = xmldoc.createElement('token')
# root_element.setAttribute('id', 'id_value')  # attribute, value
# root_element.setAttribute('canonical', 'canonical_value')
# xmldoc.appendChild(root_element)

# for user in range(len(users_list)):

#     # create child element
#     productChild = xmldoc.createElement('variant')

#     # insert user data into element
#     productChild.setAttribute('base', users_list[user])
#     root_element.appendChild(productChild)    

    
# xml_str = xmldoc.toprettyxml(indent ="\t")

# # save file
# save_path_file = "gfg.xml"

# with open(save_path_file, "w") as f:
#     f.write(xml_str)


In [35]:
dict_id_canonicals

defaultdict(list,
            {'obo:FMA_77108': ['Segment of lateral medullary reticular complex'],
             'NDDO:NDDO_20007183': ['Left_Cun_cuneus Datatype'],
             'obo:NBO_0000034': ['sexual activity'],
             'obo:NBO_0000446': ['visual pursuit'],
             'NDDO:NDDO_20000707': ['Structural MRI image'],
             'obo:NBO_0000351': ['body coordination'],
             'obo:IAO_0000219': ['denotes'],
             'NDDO:NDDO_10000792': ['Late Mild Cognitive Impairment (LMCI)'],
             'obo:OBI_0000764': ['GenePattern k-means clustering'],
             'NDDO:NDDO_20006778': ['Right_Pallidum Datatype'],
             'obo:NBO_0000208': ['classical conditioning'],
             'obo:FMA_81154': ['Brodmann area 50'],
             'obo:NPT_0011021': ['MMSE month assay design'],
             'NDDO:NDDO_10001281': ['UCSF ICV'],
             'obo:NBO_0000412': ['pupillary light reflex'],
             'obo:NPT_0010072': ['linguistic letter fluency cognitive process

In [36]:
dict_id_canonicals.get("obo:BFO_0000117")
# what if one canonical is the same as the variant?

['has occurrent part', 'has occurent part']

### need to change the token_ids and ids_iri

In [37]:
## We have two dictionaries: 1) the {token_id:[canonical]}, 2) the {token_id:[variant1, variant2]}
## use the token_id to link canonical and variants, and then create the <token id, canonical, variants> set.


# create file
xmldoc = minidom.Document()

# creat root element
root_element = xmldoc.createElement('synonym')
xmldoc.appendChild(root_element)

# combine token ids from both <AbbreviatedIRI> and <IRI> structures
token_ids.extend(list(set(token_id_iri)))   # don't use ids_iri, not every canonical has m"eta:prefixiri" attribute, use in the token ids in the id_canonical dictionary

for token_id in token_ids:
    productChild = xmldoc.createElement('token')
    productChild.setAttribute('id', token_id)  # attribute, value
    
    canonical = dict_id_canonicals.get(token_id, None)
    if canonical is not None:
        productChild.setAttribute('canonical', canonical[0])  # take the first canonical if one token_id has multiple canonicals
        variant_list = canonical[1:]  # add the rest of the canonicals as the variant
    else:
#         print(token_id)   # the print statement prints nothing, it won't get into the else statement 
        continue
        
    root_element.appendChild(productChild) 

    variant = dict_id_variants[token_id]
    variant_list.extend(variant)
    
    if variant_list is not None:
        variant_list = list(set(variant_list)) # add the canonicals to the variant, eliminate the duplications
    else:
        variant_list = variant  # when the token_id only have one canonical and have no variants
    
    variant_list.append(canonical[0])  # the canonical has to be in the variant; I don't want to mess up with the above codes so I add the canonical in the end of the variant list
    
    variant_list = list(set(variant_list))  # avoid duplications
    
    for user in range(len(variant_list)):

        # create child element
        product_grandChild = xmldoc.createElement('variant')

        # insert user data into element
        product_grandChild.setAttribute('base', variant_list[user])
        productChild.appendChild(product_grandChild)    

    
xml_str = xmldoc.toprettyxml(indent ="\t")

# save file
dict_path = "/Users/yidesdo21/Projects/inputs/dictionary/"
save_path_file = "nio_case.xml"   # have to be very careful for this, this is not the final xml dictionary 

with open(dict_path+save_path_file, "w") as f:
    f.write(xml_str)


In [38]:
len(token_ids)

3755

## combine ado.xml and nio*.xml together

<!-- <token id="obo:OBI_0002555" canonical="age group inclusion criterion">
    <variant base="AlzheimerOntology (ADO):age_eligibility"/>    <----- better be "age eligibility"
    <variant base="age group inclusion criterion"/>
</token>
<token id="obo:OBI_0002561" canonical="sex inclusion criterion">
    <variant base="AlzheimerOntology (ADO):gender_eligibility"/> <----- better be "gender eligibility"
    <variant base="sex inclusion criterion"/>
</token> -->